<a href="https://colab.research.google.com/github/Mohona77/Audio-Classification-SER-/blob/main/BanglaSER_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.9/208.9 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [4]:
%%capture
!pip install gdown

In [5]:
import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [6]:
%%capture
!gdown --id 1fZoWnL0neesTfj3l1kQRNN5O0Sfm3Bjl
time.sleep(5)

!unzip /content/BanglaSER.zip

In [7]:
import os
import librosa
import json
import glob
from sklearn.model_selection import train_test_split

# Emotion mapping based on your dataset description
emotion_map = {
    1: "Happy",
    2: "Sad",
    3: "Angry",
    4: "Surprise",
    5: "Neutral"
}

def parse_filename(filename):
    """Parses the filename to extract the emotion label."""
    parts = filename.split('-')
    if len(parts) >= 3:  # Adjust based on your filename structure
        emotion_label = int(parts[2])  # Assumes the 3rd part is the emotion label
        return emotion_map.get(emotion_label, "Unknown")
    return None

def calculate_audio_length(file_path):
    """Calculates the length of an audio file in seconds."""
    audio, sr = librosa.load(file_path, sr=None)
    return librosa.get_duration(y=audio, sr=sr)

def create_filtered_data(directory):
    """Creates a minimal JSON data structure for audio files in a directory, excluding 'Surprise' emotion."""
    json_data = []

    # Using glob to find all WAV files in the directory
    audio_files = glob.glob(os.path.join(directory, '**/*.wav'), recursive=True)

    for file_path in audio_files:
        filename = os.path.basename(file_path)
        audio_id = os.path.splitext(filename)[0]
        emotion_label = parse_filename(filename)

        # Exclude files with emotion "Surprise"
        if emotion_label and emotion_label != "Surprise":
            audio_length = calculate_audio_length(file_path)

            # Create minimal data entry
            data_entry = {
                "id": audio_id,
                "emotion": emotion_label,
                "length": audio_length,
                "wav": file_path
            }
            json_data.append(data_entry)

    return json_data

def split_and_save_data(json_data, seed=32):
    """Splits the data into train and test sets and saves them to separate JSON files."""
    train, test = train_test_split(json_data, test_size=0.2, random_state=seed)

    # Save train data
    with open('train_data.json', 'w') as train_file:
        json.dump(train, train_file, indent=4)

    # Save test data
    with open('test_data.json', 'w') as test_file:
        json.dump(test, test_file, indent=4)



In [8]:
# Directory path for your audio files
directory_path = '/content/t9h6p943xy-5/BanglaSER'

# Step 1: Create filtered data directly
print("Creating filtered data...")
filtered_data = create_filtered_data(directory_path)

# Step 2: Split and save data to train and test JSON files
print("Splitting and saving data into train and test JSON files...")
split_and_save_data(filtered_data)

print("Train and test JSON files created successfully.")


Creating filtered data...
Splitting and saving data into train and test JSON files...
Train and test JSON files created successfully.


In [11]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/t9h6p943xy-5/BanglaSER/train_data.json",'r') as f:
    train_data = json.load(f)

with open("/content/t9h6p943xy-5/BanglaSER/test_data.json",'r') as f:
    test_data = json.load(f)

In [12]:

train_data

[{'id': '03-01-05-01-03-03-26',
  'emotion': 'Neutral',
  'length': 3.5782766439909297,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 26/03-01-05-01-03-03-26.wav'},
 {'id': '03-01-05-01-01-03-03',
  'emotion': 'Neutral',
  'length': 3.353061224489796,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 03/03-01-05-01-01-03-03.wav'},
 {'id': '03-01-05-01-02-01-28',
  'emotion': 'Neutral',
  'length': 3.7533786848072563,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 28/03-01-05-01-02-01-28.wav'},
 {'id': '03-01-05-01-01-01-31',
  'emotion': 'Neutral',
  'length': 3.6603854875283446,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 31/03-01-05-01-01-01-31.wav'},
 {'id': '03-01-05-01-02-02-06',
  'emotion': 'Neutral',
  'length': 3.669863945578231,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 06/03-01-05-01-02-02-06.wav'},
 {'id': '03-01-03-02-01-02-06',
  'emotion': 'Angry',
  'length': 3.4548125,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 06/03-01-03-02-01-02-06.wav'},
 {'id': '03

In [13]:
test_data

[{'id': '03-01-01-01-02-03-10',
  'emotion': 'Happy',
  'length': 3.150166666666667,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 10/03-01-01-01-02-03-10.wav'},
 {'id': '03-01-02-01-02-03-24',
  'emotion': 'Sad',
  'length': 3.5460833333333333,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 24/03-01-02-01-02-03-24.wav'},
 {'id': '03-01-02-01-01-01-12',
  'emotion': 'Sad',
  'length': 3.2226666666666666,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 12/03-01-02-01-01-01-12.wav'},
 {'id': '03-01-02-01-01-02-02',
  'emotion': 'Sad',
  'length': 3.5342403628117913,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 02/03-01-02-01-01-02-02.wav'},
 {'id': '03-01-05-01-02-02-02',
  'emotion': 'Neutral',
  'length': 3.0736961451247167,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 02/03-01-05-01-02-02-02.wav'},
 {'id': '03-01-01-01-01-02-23',
  'emotion': 'Happy',
  'length': 3.2354195011337867,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 23/03-01-01-01-01-02-23.wav'},
 {'id': '03-01-

In [14]:

train_samples_list = [
    {**sample_data, "id": i} for i, sample_data in enumerate(train_data)
]
train_dataset = Dataset.from_list(train_samples_list)

In [16]:
test_samples_list = [
    {**sample_data, "id": i} for i, sample_data in enumerate(test_data)
]
test_dataset = Dataset.from_list(test_samples_list)

In [17]:
dataset = DatasetDict({
    'train' : train_dataset,
    'test' : test_dataset,
    'dev' : test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'emotion', 'length', 'wav'],
        num_rows: 928
    })
    test: Dataset({
        features: ['id', 'emotion', 'length', 'wav'],
        num_rows: 233
    })
    dev: Dataset({
        features: ['id', 'emotion', 'length', 'wav'],
        num_rows: 233
    })
})

In [ ]:
def add_base_path(example):
    # Check if base path is already in the file path
    if not example['wav'].startswith('/content/t9h6p943xy-5/BanglaSER/'):
        example['wav'] = '/content/t9h6p943xy-5/BanglaSER/' + example['wav']  # Adjust column name if it's not 'wav'
    return example

dataset = dataset.map(add_base_path, batched=False)

Map:   0%|          | 0/928 [00:00<?, ? examples/s]

Map:   0%|          | 0/233 [00:00<?, ? examples/s]

Map:   0%|          | 0/233 [00:00<?, ? examples/s]

In [18]:
import librosa
def load_audio(example):
  audio_array, sample_rate = librosa.load(example['wav'], sr=16000)
  example["audio_array"] = audio_array
  example["sample_rate"] = sample_rate
  return example

In [19]:
dataset = dataset.map(load_audio, batched=False)
dataset

Map:   0%|          | 0/928 [00:00<?, ? examples/s]

Map:   0%|          | 0/233 [00:00<?, ? examples/s]

Map:   0%|          | 0/233 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'emotion', 'length', 'wav', 'audio_array', 'sample_rate'],
        num_rows: 928
    })
    test: Dataset({
        features: ['id', 'emotion', 'length', 'wav', 'audio_array', 'sample_rate'],
        num_rows: 233
    })
    dev: Dataset({
        features: ['id', 'emotion', 'length', 'wav', 'audio_array', 'sample_rate'],
        num_rows: 233
    })
})